# Docling image parsing example
![alt text](images/Docling%20qs%20earch.png)

based on this https://docling-project.github.io/docling/examples/pictures_description_api/


### 0. Preperation

install docling and split the document and managable sizes (for demo purposes only)

In [ ]:
!pip install docling

In [ ]:
# split pdf using python script
# this snippet includes an interesting diagrtam
!python pdf_splitter_tool.py ../../00_setup/01_raw_data/db2_data_mov_115.pdf 19 20 pdf_snippet_with_diagram_0.pdf

### 1. Basic docling parsing

This ignores images and just leaves a < image > marker in the parsed markdown

In [ ]:
from docling.document_converter import DocumentConverter

source = "pdf_snippet_with_diagram_0.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())  # output: "## Docling Technical Report[...]"
# store md in a file with the same name as the pdf + basic_pipeline.md
with open(f"{source.split('.')[0]}_1_basic_pipeline.md", "w") as f:
    f.write(result.document.export_to_markdown())

### 2. Setting up vision based parsing

This will send all detected images to a multimodel LLM in watsonx

In [ ]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    PictureDescriptionApiOptions,
)
from docling.document_converter import DocumentConverter, PdfFormatOption
import requests
from docling_core.types.doc import PictureItem
from dotenv import load_dotenv

In [ ]:
def watsonx_vlm_options():
    load_dotenv()
    api_key = "..."
    project_id = "0701487d-97b1-4583-8980-b3cbfcac032a"

    def _get_iam_access_token(api_key: str) -> str:
        res = requests.post(
            url="https://iam.cloud.ibm.com/identity/token",
            headers={
                "Content-Type": "application/x-www-form-urlencoded",
            },
            data=f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={api_key}",
        )
        res.raise_for_status()
        api_out = res.json()
        print(f"{api_out=}")
        return api_out["access_token"]

    options = PictureDescriptionApiOptions(
        url="https://us-south.ml.cloud.ibm.com/ml/v1/text/chat?version=2023-05-29",
        params=dict(
            model_id="meta-llama/llama-3-2-90b-vision-instruct",
            project_id=project_id,
            parameters=dict(
                max_new_tokens=400,
            ),
        ),
        headers={
            "Authorization": "Bearer " + _get_iam_access_token(api_key=api_key),
        },
        prompt="""You are creating a description of the image to make it usable in a RAG pipeline. 
                    Describe the image in a few sentences. 
                    If it is a diagram, make sure to describe the relationships between the elements.
                    Be consise and accurate.""",
        timeout=60,
    )
    return options

In [ ]:
def main(source):

    pipeline_options = PdfPipelineOptions(
        enable_remote_services=True  # <-- this is required!
    )
    pipeline_options.do_picture_description = True
    pipeline_options.picture_description_options = watsonx_vlm_options()

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options,
            )
        }
    )
    result = doc_converter.convert(source)
    

    for element, _level in result.document.iterate_items():
        if isinstance(element, PictureItem):
            print(
                f"Picture {element.self_ref}\n"
                f"Caption: {element.caption_text(doc=result.document)}\n"
                f"Annotations: {element.annotations}"
            )
    with open(f"{source.split('.')[0]}_2_advanced_pipeline.md", "w") as f:
        f.write(result.document.export_to_markdown())
        
if __name__ == "__main__":
    main(source)
    